# Transformer 模型家族

自2017年推出以来，[原始的 Transformer 模型](https://arxiv.org/abs/1706.03762)（可以通过[《注释版 Transformer》](http://nlp.seas.harvard.edu/2018/04/03/attention.html)这篇博客文章获取更易懂的技术介绍）启发了许多新模型，这些模型的应用范围已经超出了自然语言处理（NLP）任务。这些模型包括[预测蛋白质折叠结构](https://huggingface.co/blog/deep-learning-with-proteins)，[训练猎豹奔跑](https://huggingface.co/blog/train-decision-transformers)，以及[时间序列预测](https://huggingface.co/blog/time-series-transformers)。有这么多的 Transformer 变体，人们很容易忽略整体情况。这些模型的共同点是它们都基于原始的 Transformer 架构。有些模型只使用了编码器或解码器，而有些则同时使用了两者。这为我们提供了一个有用的分类方法，可以帮助理解不同模型之间的高级差异，也帮助你理解之前未接触过的 Transformer 模型。

如果你不熟悉原始的 Transformer 模型或需要复习，可以查看 Hugging Face 课程的[《Transformer 是如何工作的](https://huggingface.co/course/chapter1/4?fw=pt)》章节。

## 计算机视觉

### 卷积网络

长期以来，卷积网络（CNN）一直是计算机视觉任务的主流范式，直到 [Vision Transformer](https://arxiv.org/abs/2010.11929) 展示了它的可扩展性和高效性。即使如此，CNN 的一些优秀特性（如平移不变性）对于某些任务来说仍然非常强大，因此一些 Transformer 采用了卷积结构。相反，[ConvNeXt](model_doc/convnext) 用 Transformer 的设计理念来改进 CNN。例如，ConvNeXt 使用非重叠滑动窗口将图像划分为补丁，并使用更大的卷积核以增加全局感受野。ConvNeXt 还引入了几种层设计选择，以提高内存效率和性能，使其与 Transformer 竞争。

### 视觉编码器

[Vision Transformer (ViT)](model_doc/vit) 打开了计算机视觉任务中不使用卷积的大门。ViT 使用标准的 Transformer 编码器，但其主要突破在于如何处理图像。它将图像分割为固定大小的补丁，并使用这些补丁创建嵌入，就像将句子分割为标记一样。ViT 利用 Transformer 的高效架构，在训练所需的资源较少的情况下，展示了与当时的 CNN 竞争的结果。随后，其他视觉模型也出现了，可以处理密集视觉任务（如分割和检测）。

其中一个模型是 [Swin](model_doc/swin) Transformer。它类似于 CNN，从较小的补丁构建分层特征图，并将其与相邻补丁合并到更深层。注意力机制仅在局部窗口内计算，并且在注意力层之间移动窗口，以帮助模型学习。由于 Swin Transformer 可以生成分层特征图，因此它适用于密集预测任务（如分割和检测）。[SegFormer](model_doc/segformer) 也使用 Transformer 编码器来构建分层特征图，但其添加了一个简单的多层感知器（MLP）解码器，用于组合所有特征图并进行预测。

其他视觉模型，如 BeIT 和 ViTMAE，借鉴了 BERT 的预训练目标。[BeIT](model_doc/beit) 通过 _masked image modeling (MIM)_ 进行预训练：图像补丁被随机遮蔽，并且图像也被分词化为视觉标记。BeIT 被训练以预测遮蔽补丁对应的视觉标记。[ViTMAE](model_doc/vitmae) 的预训练目标类似，但它必须预测像素而不是视觉标记。特别之处在于，75% 的图像补丁被遮蔽！解码器从遮蔽的标记和编码的补丁中重建像素。预训练后，解码器被丢弃，编码器准备用于下游任务。

### 视觉解码器

仅使用解码器的视觉模型较为罕见，因为大多数视觉模型依赖编码器来学习图像表示。但在像图像生成这样的用例中，解码器是一个自然的选择，我们从文本生成模型（如 GPT-2）中已经看到了这一点。[ImageGPT](model_doc/imagegpt) 与 GPT-2 使用相同的架构，但它预测的是图像中的下一个像素，而不是序列中的下一个标记。除了图像生成，ImageGPT 还可以微调用于图像分类。

### 视觉编码器-解码器

视觉模型通常使用编码器（也称为主干）提取重要的图像特征，然后将其传递给 Transformer 解码器。[DETR](model_doc/detr) 使用预先训练的主干，但还使用完整的 Transformer 编码器-解码器架构进行目标检测。编码器学习图像表示，并在解码器中将其与目标查询（每个目标查询是聚焦于图像中某个区域或目标的嵌入）结合。DETR 预测每个目标查询的边界框坐标和类别标签。

## 自然语言处理

### NLP 编码器

[BERT](model_doc/bert) 是一个仅使用编码器的 Transformer，它通过随机遮蔽输入中的某些标记来避免“作弊”。预训练目标是根据上下文预测被遮蔽的标记。这使 BERT 能够充分利用左右上下文，以帮助它学习更深入和丰富的输入表示。然而，BERT 的预训练策略仍有改进的空间。[RoBERTa](model_doc/roberta) 通过引入新的预训练方法来改进这一点，包括更长时间的训练和更大的批量训练，每个 epoch 随机遮蔽标记（而不仅仅是预处理阶段一次遮蔽），并删除了下一个句子预测目标。

改进性能的主要策略是增加模型规模。但训练大型模型在计算上代价高昂。一种减少计算成本的方法是使用更小的模型，如 [DistilBERT](model_doc/distilbert)。DistilBERT 使用[知识蒸馏](https://arxiv.org/abs/1503.02531)——一种压缩技术——创建一个更小版本的 BERT，同时保持其绝大部分的语言理解能力。

然而，大多数 Transformer 模型继续增加参数数量，导致出现了新的模型，专注于提高训练效率。[ALBERT](model_doc/albert) 通过两种方式减少内存消耗：将较大的词汇表嵌入分为两个较小的矩阵，并允许多层共享参数。[DeBERTa](model_doc/deberta) 引入了一种解耦的注意力机制，将单词和位置分别编码为两个向量。注意力从这些独立的向量计算，而不是从单个包含单词和位置嵌入的向量计算。[Longformer](model_doc/longformer) 也专注于使注意力效率更高，特别是处理较长序列的文档。它使用局部窗口注意（注意力仅计算每个标记周围的固定窗口大小）和全局注意力（仅用于特定的任务标记，如分类中的 `[CLS]`）来创建稀疏注意力矩阵而不是完全注意力矩阵。

### NLP 解码器

[GPT-2](model_doc/gpt2) 是一个仅使用解码器的 Transformer，它预测序列中的下一个单词。它遮蔽右侧的标记，以防止模型“作弊”。通过预训练大量文本，GPT-2 变得非常擅长生成文本，即使这些文本有时是不准确或不真实的。但 GPT-2 缺乏 BERT 预训练中的双向上下文，这使其不适合某些任务。[XLNET](model_doc/xlnet) 结合 BERT 和 GPT-2 的预训练目标，使用排列语言建模目标（PLM），使之能够学习双向上下文。

在 GPT-2 之后，语言模型变得更大，现在被称为 _大型语言模型（LLMs）_。LLMs 如果在足够大的数据集上预训练，可以展示出少量或零样本学习的能力。[GPT-J](model_doc/gptj) 是一个具有 6B 参数的 LLM，并在 400B 个标记上进行了训练。GPT-J 之后是 [OPT](model_doc/opt)，这是一系列仅使用解码器的模型，最大的模型有 175B 参数，并在 180B 个标记上进行了训练。[BLOOM](model_doc/bloom) 也在同一时间发布，最大的模型有 176B 参数，并在 46 种语言和 13 种编程语言的 366B 个标记上进行了训练。

### NLP 编码器-解码器

[BART](model_doc/bart) 保持了原始的 Transformer 架构，但修改了预训练目标：通过 _文本填充_ 腐蚀，将一些文本片段替换为单个 `mask` 标记。解码器预测未腐败的标记（未来标记被遮蔽），并使用编码器的隐状态来帮助它。[Pegasus](model_doc/pegasus) 类似于 BART，但它遮蔽的是整个句子而不是文本片段。除了掩码语言建模，Pegasus 还通过句子间隙生成（GSG）进行预训练。GSG 目标在于屏蔽文书中重要的整个句子，用 `mask` 标记替换它们。解码器必须根据剩余的句子生成输出。[T5](model_doc/t5) 是一个更独特的模型，它将所有 NLP 任务转换为文本到文本的问题，使用特定的前缀。例如，前缀 `Summarize:` 表示摘要任务。T5 通过监督（GLUE 和 SuperGLUE）训练和自我监督训练（随机样本并丢弃 15% 的标记）进行预训练。

## 音频

### 音频编码器

[Wav2Vec2](model_doc/wav2vec2) 使用 Transformer 编码器直接从原始音频波形中学习语音表示。它通过对比任务进行预训练，以确定真实语音表示和其他虚假表示之间的差异。[HuBERT](model_doc/hubert) 与 Wav2Vec2 类似，但训练过程不同。目标标签是通过聚类步骤创建的，其中相似的音频片段被分配到一个聚类，该聚类成为隐藏单元。隐藏单元被映射到嵌入以进行预测。

### 音频编码器-解码器

[Speech2Text](model_doc/speech_to_text) 是一个用于自动语音识别（ASR）和语音翻译的语音模型。该模型接受从音频波形中提取的对数梅尔滤波器组特征，并自回归地预训练以生成转录或翻译。[Whisper](model_doc/whisper) 也是一个 ASR 模型，但与许多其他语音模型不同，它是在巨大的✨标注✨音频转录数据上预训练的，因此可以在零样本情况下表现良好。数据集中很大一部分是非英语语言，这意味着 Whisper 也可以用于低资源语言。在结构上，Whisper 类似于 Speech2Text。音频信号被转换为由编码器编码的对数梅尔谱图。解码器根据编码器的隐状态和之前的标记自回归地生成转录。

## 多模态

### 多模态编码器

[VisualBERT](model_doc/visual_bert) 是一个用于视觉-语言任务的多模态模型，发布于 BERT 之后不久。它结合了 BERT 和预训练的对象检测系统，将图像特征提取为视觉嵌入，并与文本嵌入一起传递给 BERT。VisualBERT 根据未遮蔽的文本和视觉嵌入预测遮蔽的文本，并预测文本是否与图像对齐。随着 ViT 的发布，[ViLT](model_doc/vilt) 在其架构中采用了 ViT，因为这样更容易获得图像嵌入。图像嵌入与文本嵌入共同处理。从那时起，ViLT 通过图像文本匹配、掩码语言建模和整体词汇遮蔽进行预训练。

[CLIP](model_doc/clip) 采用不同的方法，对 (`image`, `text`) 对进行预测。图像编码器（ViT）和文本编码器（Transformer）共同在包含 4 亿个 (`image`, `text`) 对的数据集上进行预训练，以最大化图像和文本嵌入之间的相似性。预训练后，可以使用自然语言指导 CLIP 从图像预测文本或反之亦然。[OWL-ViT](model_doc/owlvit) 在 CLIP 的基础上进行了扩展，使用 CLIP 作为其零样本目标检测的主干。预训练后，添加了一个目标检测头，对 (`class`, `bounding box`) 对进行预测。

### 多模态编码器-解码器

光学字符识别（OCR）是文本识别的长期任务，通常涉及多个组件来理解图像并生成文本。[TrOCR](model_doc/trocr) 使用端到端的 Transformer 简化了这一过程。编码器是一个 ViT 风格的模型，用于图像理解，并将图像处理为固定大小的补丁。解码器接受编码器的隐状态并自回归地生成文本。[Donut](model_doc/donut) 是一个更通用的视觉文档理解模型，不依赖于基于 OCR 的方法。它使用 Swin Transformer 作为编码器和多语言 BART 作为解码器。Donut 通过预测图像和文本注释中的下一个词进行预训练。解码器根据提示生成标记序列。每个下游任务都有一个特殊标记。例如，文档解析有一个特殊的 `parsing` 标记，结合编码器的隐状态将文档解析为结构化的输出格式（JSON）。

## 强化学习

### 强化解码器

Decision 和 Trajectory Transformer 将状态、动作和奖励视为序列建模问题。[Decision Transformer](model_doc/decision_transformer) 生成一系列动作，使未来期望的回报最大化，基于回报到目前为止、过去的状态和动作。对于最后 _K_ 个时间步，每个模态被转换为标记嵌入，并由 GPT 样式模型处理以预测未来的动作标记。[Trajectory Transformer](model_doc/trajectory_transformer) 也将状态、动作和奖励标记化，并使用 GPT 架构处理。与 Decision Transformer 不同，它专注于奖励条件，Trajectory Transformer 通过束搜索生成未来的动作。